# LAFD Response Time

## Background:

Back in early 2012, The Los Angeles Fire Department admitted to the [Los Angeles Times](https://latimesblogs.latimes.com/lanow/2012/03/la-fire-department-admits-exaggerating-response-times.html) that they exaggerated the data to make it appear that firefighters were responding to emergencies quicker than they actually were. The false data reported that in 2008 the department responded to emergencies within five minutes about 86 percent of the time. It turns out the LAFD categorized 6 minute responses the same as 5 minute responses. Apparently the corrected data shows that the department actually met the five-minute benchmark only 64 percent of the time in 2008. In the LA Times report mentioned above, it mentions the "Federal guidelines call for first responders to arrive on scene in under five minutes 90% of the time."

According to the city wide response metrics from January to March of 2019, found on the [LAFD's website](https://www.lafd.org/fsla/stations-map), the average EMS (emergency medical service) response time is 6 minutes 36 seconds. 

## Problem Statement:

We will use two datas from the Los Angeles City [website](https://data.lacity.org/A-Safe-City/All-Stations-Response-Metrics/kszm-sdw4), LAFD's response metric and Census Data of Los Angeles neighborhoods, in order to explore the emergency metrics and census data for each of the LA neighborhoods. The main objectives for this project are listed as follows:

**1) We want to analyze if certain locations (districts/neighborhoods) within the Los Angeles City has any affect on the time it takes for the LAFD respondents to arrive at the scene. In this project we strive to investigate whether certain factors such as, the poverty rate, average income, or a majority ethnic group of a location in LA has any correlation to the average response time from the LAFD.**

**2) We will build a regression model to predict the response time of the LAFD when it comes to EMS reports for a particular location in the City of Los Angeles. Model performance will be guided by RMSE, and the model should improve the baseline by as much as it can.** 
 

### Dataset:

[Source](https://data.lacity.org/A-Safe-City/LAFD-Response-Metrics-Raw-Data/n44u-wxe4)
* LAFD Response Metrics - Raw Data
    - Data last updated in April 8, 2019

[Source](https://data.lacity.org/A-Safe-City/All-Stations-Response-Metrics/kszm-sdw4)
* Archived response Time metrics for all LAFD stations
    - Contains all stations response metrics from 2011 to 2017
    
[Source](
  
[Source](https://data.lacity.org/A-Livable-and-Sustainable-City/Census-Data-by-Neighborhood-Council/nwj3-ufba)
* Census 2010 population/demographic data approximated from block groups to LA Neighborhood Councils
    - Data last updated in April 2018


In [2]:
ls

Capstone_part_2.ipynb  data/


In [6]:
import pandas as pd

In [7]:
fire_station = pd.read_csv('./data/FireStations.csv')
fire_station.shape

(106, 14)

In [8]:
census = pd.read_csv('./data/Census_Data_by_Neighborhood_Council.csv')
census.shape

(97, 13)

In [9]:
response = pd.read_csv('./data/All_Stations_Response_Metrics.csv')
response.shape

(5610, 9)

In [10]:
response.head()

,YEAR,DISTRICT,MONTH,AVERAGE LAFD TURNOUT TIME IN DISTRICT - Non-EMS,AVERAGE LAFD TURNOUT TIME IN DISTRICT - EMS,AVERAGE LAFD TRAVEL TIME IN DISTRICT - Non-EMS,AVERAGE LAFD TRAVEL TIME IN DISTRICT - EMS,INCIDENT COUNT IN DISTRICT - Non-EMS,INCIDENT COUNT IN DISTRICT - EMS
0,2013,1,Jan,0:01:22,0:01:10,0:04:39,0:04:17,45,183
1,2013,1,Feb,0:01:19,0:01:12,0:03:47,0:04:16,38,165
2,2013,1,Mar,0:01:20,0:01:13,0:04:05,0:04:17,44,174
3,2013,1,Apr,0:01:18,0:01:10,0:04:59,0:04:03,45,173
4,2013,1,May,0:01:16,0:01:16,0:04:30,0:04:16,53,234


In [11]:
response.YEAR.value_counts()

2015    1326
2014    1326
2013    1326
2016    1326
2017     306
Name: YEAR, dtype: int64

In [15]:
fire_station.head()

,the_geom,FS_CD,SHP_ADDR,ADDRESS,ZIP,DEPTNAME,HSENO,PREF_DIR,STNAME,STTYPE,SUFF_DIR,UNITDESC,X_COORD,Y_COORD
0,POINT (-118.30847478565374 34.08375012794097),52,Fire Station 52,4957 MELROSE AVE.,90029,FIRE,4957,NaN,MELROSE,AVE,NaN,NaN,6.468261e+06,1.853004e+06
1,POINT (-118.28654226892745 34.07708277350275),6,Fire Station 6,326 N. VIRGIL AVE.,90004,FIRE,326,N,VIRGIL,AVE,NaN,NaN,6.474895e+06,1.850558e+06
2,POINT (-118.42037285259569 34.04404529506214),92,Fire Station 92,10556 W. PICO BLVD,90064,FIRE,10556,W,PICO,BLVD,NaN,NaN,6.434222e+06,1.838657e+06
3,POINT (-118.29120647521931 33.968767788084904),57,Fire Station 57,7800 S. VERMONT AVE.,90044,FIRE,7800,S,VERMONT,AVE,NaN,NaN,6.473369e+06,1.811144e+06
4,POINT (-118.5100228348254 34.192900556972354),100,Fire Station 100,6751 LOUISE AVE,91406,FIRE,6751,NaN,LOUISE,AVE,NaN,NaN,6.407435e+06,1.892974e+06


In [16]:
census.head()

,NC_Name,Total Population,White_pop,Black_pop,Ameri_es_pop,Asian_pop,Hawn_pi_pop,Hispanic_pop,Other_pop,Multi_pop,In_Poverty,Owner_occ,Renter_occ
0,ARLETA NC,34932.84,2882.67,409.67,67.64,4061.31,34.96,NaN,37.58,245.05,34700.56,5590.27,2159.94
1,ARROYO SECO NC,21711.47,4640.35,782.76,89.51,3215.37,25.38,NaN,66.98,263.12,21582.85,4138.14,3235.61
2,ATWATER VILLAGE NC,11385.40,3450.70,127.80,0.15,2379.08,71.85,NaN,0.37,252.76,11345.53,1877.30,2902.89
3,BEL AIR-BEVERLY CREST NC,26789.14,22024.61,349.63,0.02,1787.79,0.01,NaN,156.92,946.28,26623.97,8939.21,1943.18
4,BOYLE HEIGHTS NC,81900.56,1562.20,640.97,63.63,2266.59,73.99,NaN,90.30,98.50,81144.31,4743.28,16001.77


In [17]:
fire_station.head()

,the_geom,FS_CD,SHP_ADDR,ADDRESS,ZIP,DEPTNAME,HSENO,PREF_DIR,STNAME,STTYPE,SUFF_DIR,UNITDESC,X_COORD,Y_COORD
0,POINT (-118.30847478565374 34.08375012794097),52,Fire Station 52,4957 MELROSE AVE.,90029,FIRE,4957,NaN,MELROSE,AVE,NaN,NaN,6.468261e+06,1.853004e+06
1,POINT (-118.28654226892745 34.07708277350275),6,Fire Station 6,326 N. VIRGIL AVE.,90004,FIRE,326,N,VIRGIL,AVE,NaN,NaN,6.474895e+06,1.850558e+06
2,POINT (-118.42037285259569 34.04404529506214),92,Fire Station 92,10556 W. PICO BLVD,90064,FIRE,10556,W,PICO,BLVD,NaN,NaN,6.434222e+06,1.838657e+06
3,POINT (-118.29120647521931 33.968767788084904),57,Fire Station 57,7800 S. VERMONT AVE.,90044,FIRE,7800,S,VERMONT,AVE,NaN,NaN,6.473369e+06,1.811144e+06
4,POINT (-118.5100228348254 34.192900556972354),100,Fire Station 100,6751 LOUISE AVE,91406,FIRE,6751,NaN,LOUISE,AVE,NaN,NaN,6.407435e+06,1.892974e+06


In [18]:
response.DISTRICT.sort_values()

0         1
2662      1
2663      1
2664      1
3978      1
3979      1
3980      1
3981      1
3982      1
3983      1
3984      1
3985      1
3986      1
3987      1
3988      1
3989      1
1327      1
5304      1
2661      1
5305      1
2660      1
2658      1
1328      1
1329      1
1330      1
1331      1
1332      1
1333      1
1334      1
1335      1
       ... 
2649    112
3976    112
5303    112
3977    112
1324    112
5301    112
5300    112
5299    112
5298    112
5297    112
5296    112
5295    112
5294    112
5293    112
5292    112
5291    112
2651    112
1313    112
1314    112
1315    112
1316    112
1317    112
1318    112
1319    112
1320    112
1321    112
1322    112
1323    112
1325    112
5609    112
Name: DISTRICT, Length: 5610, dtype: int64

In [19]:
lafd = pd.read_csv('./data/LAFD_Response_Metrics_-_Raw_Data.csv')
lafd.shape

(490080, 11)

In [20]:
lafd.head(2)

,Randomized Incident Number,First In District,Emergency Dispatch Code,Dispatch Sequence,Dispatch Status,Unit Type,PPE Level,Incident Creation Time (GMT),Time of Dispatch (GMT),En Route Time (GMT),On Scene Time (GMT)
0,201301588315,10.0,Emergency,1,QTR,RA - ALS RESCUE AMBULANCE,EMS,08:01:04.000000,08:02:21.320320,08:04:25.938953,08:08:30.677752
1,201301588315,52.0,Emergency,2,QTR,E - ENGINE,EMS,08:00:56.000000,08:03:00.001407,08:04:30.594473,08:08:01.409394


In [44]:
lafd['First In District'].value_counts(ascending=True)

123.0        1
124.0        2
121.0        6
122.0       14
110.0       17
111.0      100
40.0       244
49.0       305
108.0      353
99.0       597
28.0       711
97.0       776
23.0       940
109.0      953
8.0       1109
24.0      1248
69.0      1271
101.0     1384
71.0      1697
44.0      1772
76.0      1890
18.0      1900
16.0      1904
17.0      1955
55.0      1996
50.0      2013
95.0      2060
112.0     2076
36.0      2077
107.0     2154
         ...  
2.0       6311
35.0      6355
3.0       6391
15.0      6434
6.0       6440
7.0       6475
51.0      6632
34.0      6671
13.0      6752
26.0      6968
68.0      7041
4.0       7181
58.0      7389
65.0      7521
98.0      7540
91.0      7558
60.0      7579
37.0      7752
61.0      8295
27.0      8924
39.0      8990
94.0      9302
89.0      9718
33.0      9828
11.0     10379
46.0     11101
57.0     12486
66.0     13018
64.0     14896
9.0      14957
Name: First In District, Length: 108, dtype: int64

In [22]:
first_in_districts = pd.read_csv('./data/LAFD_First_In_Districts.csv')
first_in_districts.shape

(102, 11)

In [23]:
first_in_districts.head(5)

,FID,BATTALION_,DIVISION_N,FIRSTIN_DI,RFSNUM,AREA_SQMI,BUREAU,SHAPE_AREA,SHAPE_LEN,Shape__Area,Shape__Length
0,1,5,3,76,76,4.620568,WEST BUREAU,0.001170,0.235831,1.751145e+07,28762.468168
1,2,14,3,86,86,3.313412,VALLEY BUREAU,0.000839,0.164303,1.256153e+07,19753.499324
2,3,2,1,55,55,2.896501,CENTRAL BUREAU,0.000733,0.157860,1.097429e+07,19094.170940
3,4,11,1,6,6,2.231758,CENTRAL BUREAU,0.000565,0.109407,8.446462e+06,13455.405544
4,5,15,3,96,96,8.052731,VALLEY BUREAU,0.002041,0.202733,3.060421e+07,24924.423164


In [38]:
fire_station.head()

,the_geom,FS_CD,SHP_ADDR,ADDRESS,ZIP,DEPTNAME,HSENO,PREF_DIR,STNAME,STTYPE,SUFF_DIR,UNITDESC,X_COORD,Y_COORD
0,POINT (-118.30847478565374 34.08375012794097),52,Fire Station 52,4957 MELROSE AVE.,90029,FIRE,4957,NaN,MELROSE,AVE,NaN,NaN,6.468261e+06,1.853004e+06
1,POINT (-118.28654226892745 34.07708277350275),6,Fire Station 6,326 N. VIRGIL AVE.,90004,FIRE,326,N,VIRGIL,AVE,NaN,NaN,6.474895e+06,1.850558e+06
2,POINT (-118.42037285259569 34.04404529506214),92,Fire Station 92,10556 W. PICO BLVD,90064,FIRE,10556,W,PICO,BLVD,NaN,NaN,6.434222e+06,1.838657e+06
3,POINT (-118.29120647521931 33.968767788084904),57,Fire Station 57,7800 S. VERMONT AVE.,90044,FIRE,7800,S,VERMONT,AVE,NaN,NaN,6.473369e+06,1.811144e+06
4,POINT (-118.5100228348254 34.192900556972354),100,Fire Station 100,6751 LOUISE AVE,91406,FIRE,6751,NaN,LOUISE,AVE,NaN,NaN,6.407435e+06,1.892974e+06


In [39]:
fire_station.SHP_ADDR.value_counts()

Fire Station 66     1
Fire Station 100    1
Fire Station 27     1
Fire Station 39     1
Fire Station 13     1
Fire Station 28     1
Fire Station 1      1
Fire Station 102    1
Fire Station 97     1
Fire Station 110    1
Fire Station 15     1
Fire Station 64     1
Fire Station 25     1
Fire Station 98     1
Fire Station 76     1
Fire Station 38     1
Fire Station 9      1
Fire Station 41     1
Fire Station 89     1
Fire Station 112    1
Fire Station 99     1
Fire Station 18     1
Fire Station 56     1
Fire Station 74     1
Fire Station 51     1
Fire Station 62     1
Fire Station 73     1
Fire Station 109    1
Fire Station 94     1
Fire Station 52     1
                   ..
Fire Station 33     1
Fire Station 85     1
Fire Station 93     1
Fire Station 60     1
Fire Station 59     1
Fire Station 68     1
Fire Station 83     1
Fire Station 34     1
Fire Station 24     1
Fire Station 21     1
Fire Station 67     1
Fire Station 48     1
Fire Station 96     1
Fire Station 77     1
Fire Stati